In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from tensorflow.keras.initializers import random_uniform
from keras import models
from keras import layers
from keras.layers import Dense, Flatten, Convolution1D, Dropout
from tensorflow.keras.optimizers import SGD


In [ ]:
np.random.seed(42)
scaler = MinMaxScaler()

def normalize_2d(matrix):
    norm = np.linalg.norm(matrix)
    print(norm)
    matrix = matrix/norm
    return matrix


X11 = pd.read_csv('tamil_prosody_male_featureset.csv').dropna().iloc[:,:-1]
X12 = pd.read_csv('tamil_prosody_female_featureset.csv').dropna().iloc[:,:-1]
X21 = pd.read_csv('kannada_prosody_male_featureset.csv').dropna().iloc[:,:-1]
X22 = pd.read_csv('kannada_prosody_female_featureset.csv').dropna().iloc[:,:-1]
X31 = pd.read_csv('telugu_prosody_male_featureset.csv').dropna().iloc[:,:-1]
X32 = pd.read_csv('telugu_prosody_female_featureset.csv').dropna().iloc[:,:-1]

X1 = pd.concat([X11, X12], axis = 0)
X2 = pd.concat([X21, X22], axis = 0)
X3 = pd.concat([X31, X32], axis = 0)

y1 = np.zeros(len(X1))
y2 = np.ones(len(X2))
y3 = np.ones(len(X3)) * 2

X = np.array(pd.concat([X1, X2, X3], axis = 0))
y = np.concatenate([y1, y2, y3])

scaler.fit(X)
X_scaled = scaler.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)

pca = PCA(n_components = 50)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

input_dimension = 103
learning_rate = 0.025
momentum = 0.85
hidden_initializer = random_uniform(seed=42)
dropout_rate = 0.2

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

model = models.Sequential()
model.add(layers.Conv1D(160, 12, kernel_initializer='he_normal', input_shape=(X_train.shape[1],X_train.shape[2]) , activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(128, 10, kernel_initializer='he_normal', activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(96, 8, kernel_initializer='he_normal', activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dropout(0.5))
model.add(Dense(32, kernel_initializer='he_normal', activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(3, activation='softmax'))

sgd = SGD(lr=learning_rate, momentum=momentum)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=sgd, metrics=['acc'])
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=128)

np.save('model/50_history.npy',history.history)
model.save('model/50_model.h5')

In [ ]:
model.summary()

In [ ]:
!pip install git+https://github.com/paulgavrikov/visualkeras --upgrade

In [ ]:
import visualkeras

visualkeras.layered_view(model = model, to_file='architecture.png').show()